# Simulation of Monty Hall Show

rules: https://en.wikipedia.org/wiki/Monty_Hall_problem

In [1]:
import random
from bokeh.plotting import figure, show, output_notebook, output_file, save,  reset_output
from bokeh.io import push_notebook
from bokeh.models import HoverTool, ColumnDataSource

In [2]:
doors = ['A', 'B', 'C']
YN_player_changes_choice = True
YN_diagnostic_printout = True

def one_game(doors, YN_player_changes_choice= True, YN_diagnostic_printout = False):
    """
    Simulate one Monty Hall game.

    Parameters:
    doors (list): A list of doors, e.g., ['A', 'B', 'C'].
    YN_player_changes_choice (bool): If True, the player always changes their initial choice after the host opens a door. If False, the player never changes their initial choice. Default is True.
    YN_diagnostic_printout (bool): If True, prints detailed diagnostic information about the game. Default is False.

    Returns:
    bool: True if the player wins the game (selects the door with the prize eventually), otherwise False.
    """
    
    # door with the prize
    door_winning = random.choice(doors)
    # what the player initialy selects
    door_selected_initially = random.choice(doors)
    # what the host can open
    candidate_doors_for_opening = set(doors) - set(door_winning) - set(door_selected_initially)
    # what the host actually opens
    door_opened = random.choice(list(candidate_doors_for_opening))
    
    # what the player selects after one door is opened
    if YN_player_changes_choice:
        door_selected_eventually = set(doors)-set(door_selected_initially)-set(door_opened)
    else:
        door_selected_eventually = set(door_selected_initially)
    
    YN_win = (door_selected_eventually == set(door_winning))
    
    if YN_diagnostic_printout:
        print(f"""winning door: {door_winning}, 
        door_selected_initially: {door_selected_initially}, 
        candidate_doors_for_opening: {candidate_doors_for_opening} -> opened: {door_opened},
        YN_player_changes_choice: {YN_player_changes_choice},
        door_selected_eventually: {door_selected_eventually},
        YN_win: {YN_win}
        """)
    return YN_win

In [3]:
# testing: never change selection
one_game(doors, YN_player_changes_choice= False, YN_diagnostic_printout = True)

winning door: A, 
        door_selected_initially: B, 
        candidate_doors_for_opening: {'C'} -> opened: C,
        YN_player_changes_choice: False,
        door_selected_eventually: {'B'},
        YN_win: False
        


False

In [4]:
# testing: always change selection
one_game(doors, YN_player_changes_choice= True, YN_diagnostic_printout = True)

winning door: B, 
        door_selected_initially: C, 
        candidate_doors_for_opening: {'A'} -> opened: A,
        YN_player_changes_choice: True,
        door_selected_eventually: {'B'},
        YN_win: True
        


True

Run the simulation:

In [5]:
CNT_TRIALS = 100   # each scenario is tried CNT_TRIALS-times

results_always_change = []
results_never_change = []

i=0
while i < CNT_TRIALS:
    
    results_always_change.append(one_game(doors, YN_player_changes_choice= True))
    results_never_change.append( one_game(doors, YN_player_changes_choice= False))
    i += 1


In [6]:
print(f""" Total results for {CNT_TRIALS} trials:
- strategy "always change": % of wins = {sum(results_always_change)/len(results_always_change)}
- strategy "never change": % of wins =  {sum(results_never_change)/ len(results_never_change)}
""")

 Total results for 100 trials:
- strategy "always change": % of wins = 0.67
- strategy "never change": % of wins =  0.34



In [7]:
# prepare counts of wins:
counts__always_change = [ sum(results_always_change[:(i+1)])  for i in range(CNT_TRIALS)]
counts__never_change =  [ sum(results_never_change[:(i+1)])   for i in range(CNT_TRIALS)]

# prepare probabilities for plotting: average the wins for first _i_ trials
probs__always_change = [ counts__always_change[i]/(i+1)  for i in range(CNT_TRIALS)]
probs__never_change =  [ counts__never_change[i] /(i+1)  for i in range(CNT_TRIALS)]

Visualize:

In [8]:
def create_plot():
    trials= list(range(1, len(probs__always_change) + 1))
    
    # Convert data to ColumnDataSource for Bokeh
    source_always_change = ColumnDataSource(data={
        'trials': trials,
        'win_percentages': probs__always_change,
        'cumulative_wins': counts__always_change
    })
    
    source_never_change = ColumnDataSource(data={
        'trials': trials,
        'win_percentages': probs__never_change,
        'cumulative_wins': counts__never_change
    })

    
    p = figure(title="Total Number of Wins in Monty Hall Problem",
               x_axis_label="Number of Trials",
               y_axis_label="Total Wins",
               width=800, height=400)
    
    # Add lines and hover tools
    p.line('trials', 'cumulative_wins', source=source_always_change,
           legend_label="Always Change", line_width=2, color="blue")
    
    p.line('trials', 'cumulative_wins', source=source_never_change,
           legend_label="Never Change", line_width=2, color="red")
    
    hover_always_change = HoverTool(renderers=[p.renderers[0]],
                                    tooltips=[
                                        ("# Trials", "@trials"),
                                        ("% of Wins", "@win_percentages")
                                    ])
    hover_never_change = HoverTool(renderers=[p.renderers[1]],
                                   tooltips=[
                                       ("# Trials", "@trials"),
                                       ("% Wins", "@win_percentages")
                                   ])
    
    p.add_tools(hover_always_change, hover_never_change)
    p.legend.location = "bottom_right"
    return(p)

In [9]:
# Save the plot to an HTML file
output_file(f"monty_hall_plot_{CNT_TRIALS}trials.html")
p = create_plot()
save(p)

'/mnt/c/Users/Vojta/D-private/programovani/python/simulace-souteze/monty_hall_plot_100trials.html'

In [10]:
# Display the plot in the notebook

reset_output()
output_notebook()
p_notebook = create_plot()
show(p_notebook)

Loading BokehJS ...